In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import BaggingRegressor

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

In [2]:
propiedades = pd.read_csv('/home/mati/Desktop/set_datos_propiedades.csv')

In [3]:
propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull()),\
                             ['place_name_encoded', 'property_type_encoded','price_aprox_usd','superficie',\
                             'Year','Month','seguridad','aire','gimnasio','cochera','pileta']]

In [4]:
propiedades.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1132495 entries, 0 to 1413024
Data columns (total 11 columns):
place_name_encoded       1132495 non-null int64
property_type_encoded    1132495 non-null int64
price_aprox_usd          1132495 non-null float64
superficie               1132495 non-null float64
Year                     1132495 non-null int64
Month                    1132495 non-null int64
seguridad                1132495 non-null bool
aire                     1132495 non-null bool
gimnasio                 1132495 non-null bool
cochera                  1132495 non-null bool
pileta                   1132495 non-null bool
dtypes: bool(5), float64(2), int64(4)
memory usage: 65.9 MB


# Bagging Regressor

In [5]:
columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [6]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [7]:
bag = BaggingRegressor()
bag.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: bag.predict(x)[0],axis = 1)
precision = bag.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
print("Precision = {:.2f} % , error = {}".format(precision, error))

Precision = 85.47 % , error = 20048219881.71454


### Ahora que tenemos una intuicion, probamos variando los parametros

In [8]:
columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [9]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

res = []

In [14]:
lista_estimators = [60,100]
lista_samples = [1.0,0.8,0.6]
lista_features = [1.0,0.7,0.5]

for estimator in lista_estimators:
    for sample in lista_samples:
        for feature in lista_features:
            bag = BaggingRegressor(n_estimators = estimator, max_samples = sample, max_features = feature)
            bag.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
            set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: bag.predict(x)[0],axis = 1)
            precision = bag.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
            error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
            res.append((estimator,sample,feature,precision,error))
            print(estimator,' - ',sample,' - ',feature)

In [15]:
for r in res:
    print ("estimator = {}, sample = {}, feature = {}, precision = {:.2f} % ,error = {}".format(r[0],r[1],r[2],r[3],r[4]))

estimator = 5, sample = 1.0, feature = 1.0, precision = 84.66 % ,error = 21165263855.557373
estimator = 5, sample = 1.0, feature = 0.7, precision = 73.85 % ,error = 36082398204.61202
estimator = 5, sample = 1.0, feature = 0.5, precision = 50.50 % ,error = 68290148675.160324
estimator = 5, sample = 0.8, feature = 1.0, precision = 85.39 % ,error = 20157385553.401493
estimator = 5, sample = 0.8, feature = 0.7, precision = 63.98 % ,error = 49695832898.52946
estimator = 5, sample = 0.8, feature = 0.5, precision = 40.39 % ,error = 82241128407.55803
estimator = 5, sample = 0.6, feature = 1.0, precision = 80.32 % ,error = 27148648359.931187
estimator = 5, sample = 0.6, feature = 0.7, precision = 62.62 % ,error = 51571637501.82258
estimator = 5, sample = 0.6, feature = 0.5, precision = 42.16 % ,error = 79794947337.227
estimator = 10, sample = 1.0, feature = 1.0, precision = 87.29 % ,error = 17534216742.50338
estimator = 10, sample = 1.0, feature = 0.7, precision = 69.06 % ,error = 42685566934.1

In [16]:
min_error = float('inf')
max_precision = 0
tupla_min_error = ()
tupla_max_precision = ()
for r in res:
    if r[3] < min_error:
        min_error = r[3]
        tupla_min_error = r
    if r[2] > max_precision:
        max_precision = r[2]
        tupla_max_precision = r
        
print("Mayor precision = estimator = {}, sample = {}, feature = {}, precision = {:.2f} % ,error = {}".\
              format(tupla_max_precision[0],tupla_max_precision[1],tupla_max_precision[2],tupla_max_precision[3],tupla_max_precision[4]))
print("Menor error = estimator = {}, sample = {}, feature = {}, precision = {:.2f} % ,error = {}".\
              format(tupla_min_error[0],tupla_min_error[1],tupla_min_error[2],tupla_min_error[3],tupla_min_error[4]))

Mayor precision = estimator = 5, sample = 1.0, feature = 1.0, precision = 84.66 % ,error = 21165263855.557373
Menor error = estimator = 5, sample = 0.8, feature = 0.5, precision = 40.39 % ,error = 82241128407.55803


# Elegimos estimator = 5, sample = 1, feature = 1

In [64]:
#Probamos varias combinaciones de columnas, la de mejor resultado fue esta

columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio','aire','pileta','cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [65]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2015) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [66]:
estimator = 5
sample = 1.0
feature = 1.0

In [67]:
bag = BaggingRegressor(n_estimators = estimator, max_samples = sample, max_features = feature)
bag.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: bag.predict(x)[0],axis = 1)
precision = bag.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
print("Precision = {:.2f} % , error = {}".format(precision, error))

Precision = 88.12 % , error = 16395593552.139215


# Calculamos los verdaderos datos

In [68]:
analizar = pd.read_csv('/home/mati/Desktop/properati_dataset_modificado.csv')

In [69]:
analizar.loc[:,'price_usd'] = analizar.loc[:,columnas].apply(lambda x: bag.predict(x)[0],axis = 1)

In [70]:
resultado = analizar.loc[:,['id','price_usd']]

In [71]:
resultado.to_csv('Bagging_Regresor_resultados.csv', index = False)

# Ahora uso lat y lon en lugar de place name

In [2]:
propiedades = pd.read_csv('/home/mati/Desktop/set_datos_propiedades.csv')

In [3]:
propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()),\
                             ['lat', 'lon', 'property_type_encoded','price_aprox_usd','superficie',\
                             'Year','Month','seguridad','aire','gimnasio','cochera','pileta']]

In [4]:
columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [6]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [7]:
bag = BaggingRegressor()
bag.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: bag.predict(x)[0],axis = 1)
precision = bag.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
print("Precision = {:.2f} % , error = {}".format(precision, error))

Precision = 95.73 % , error = 4943209822.381698


In [8]:
res = []

In [9]:
lista_estimators = [5,10,20,30,60]
lista_samples = [1.0,0.8,0.6]
lista_features = [1.0,0.7,0.5]

for estimator in lista_estimators:
    for sample in lista_samples:
        for feature in lista_features:
            bag = BaggingRegressor(n_estimators = estimator, max_samples = sample, max_features = feature)
            bag.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
            set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: bag.predict(x)[0],axis = 1)
            precision = bag.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
            error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
            res.append((estimator,sample,feature,precision,error))
            print(estimator,' - ',sample,' - ',feature)

5  -  1.0  -  1.0
5  -  1.0  -  0.7
5  -  1.0  -  0.5
5  -  0.8  -  1.0
5  -  0.8  -  0.7
5  -  0.8  -  0.5
5  -  0.6  -  1.0
5  -  0.6  -  0.7
5  -  0.6  -  0.5
10  -  1.0  -  1.0
10  -  1.0  -  0.7
10  -  1.0  -  0.5
10  -  0.8  -  1.0
10  -  0.8  -  0.7
10  -  0.8  -  0.5
10  -  0.6  -  1.0
10  -  0.6  -  0.7
10  -  0.6  -  0.5
20  -  1.0  -  1.0
20  -  1.0  -  0.7
20  -  1.0  -  0.5
20  -  0.8  -  1.0
20  -  0.8  -  0.7
20  -  0.8  -  0.5
20  -  0.6  -  1.0
20  -  0.6  -  0.7
20  -  0.6  -  0.5
30  -  1.0  -  1.0
30  -  1.0  -  0.7
30  -  1.0  -  0.5
30  -  0.8  -  1.0
30  -  0.8  -  0.7
30  -  0.8  -  0.5
30  -  0.6  -  1.0
30  -  0.6  -  0.7
30  -  0.6  -  0.5
60  -  1.0  -  1.0
60  -  1.0  -  0.7
60  -  1.0  -  0.5
60  -  0.8  -  1.0
60  -  0.8  -  0.7
60  -  0.8  -  0.5
60  -  0.6  -  1.0
60  -  0.6  -  0.7
60  -  0.6  -  0.5


In [10]:
for r in res:
    print ("estimator = {}, sample = {}, feature = {}, precision = {:.2f} % ,error = {}".format(r[0],r[1],r[2],r[3],r[4]))

estimator = 5, sample = 1.0, feature = 1.0, precision = 94.95 % ,error = 5842586565.634796
estimator = 5, sample = 1.0, feature = 0.7, precision = 93.00 % ,error = 8102370802.097091
estimator = 5, sample = 1.0, feature = 0.5, precision = 75.84 % ,error = 27975154372.92071
estimator = 5, sample = 0.8, feature = 1.0, precision = 92.69 % ,error = 8459730763.015385
estimator = 5, sample = 0.8, feature = 0.7, precision = 90.59 % ,error = 10895870717.338966
estimator = 5, sample = 0.8, feature = 0.5, precision = 81.22 % ,error = 21747450863.43158
estimator = 5, sample = 0.6, feature = 1.0, precision = 92.36 % ,error = 8847762756.185787
estimator = 5, sample = 0.6, feature = 0.7, precision = 88.11 % ,error = 13765182715.10855
estimator = 5, sample = 0.6, feature = 0.5, precision = 78.40 % ,error = 25014997141.053345
estimator = 10, sample = 1.0, feature = 1.0, precision = 95.90 % ,error = 4745785723.037289
estimator = 10, sample = 1.0, feature = 0.7, precision = 92.27 % ,error = 8953670001.35

In [11]:
min_error = float('inf')
max_precision = 0
tupla_min_error = ()
tupla_max_precision = ()
for r in res:
    if r[3] < min_error:
        min_error = r[3]
        tupla_min_error = r
    if r[2] > max_precision:
        max_precision = r[2]
        tupla_max_precision = r
        
print("Mayor precision = estimator = {}, sample = {}, feature = {}, precision = {:.2f} % ,error = {}".\
              format(tupla_max_precision[0],tupla_max_precision[1],tupla_max_precision[2],tupla_max_precision[3],tupla_max_precision[4]))
print("Menor error = estimator = {}, sample = {}, feature = {}, precision = {:.2f} % ,error = {}".\
              format(tupla_min_error[0],tupla_min_error[1],tupla_min_error[2],tupla_min_error[3],tupla_min_error[4]))

Mayor precision = estimator = 5, sample = 1.0, feature = 1.0, precision = 94.95 % ,error = 5842586565.634796
Menor error = estimator = 10, sample = 0.6, feature = 0.5, precision = 73.70 % ,error = 30450986259.416325


# Elegimos estimator = 5, sample = 1, feature = 1

In [47]:
#Probamos distintas combinaciones de columnas, esta es la que dio mejores resultados

columnas = ['superficie','lat', 'lon','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [48]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [49]:
estimator = 5
sample = 1.0
feature = 1.0

In [50]:
bag = BaggingRegressor(n_estimators = estimator, max_samples = sample, max_features = feature)
bag.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: bag.predict(x)[0],axis = 1)
precision = bag.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
print("Precision = {:.2f} % , error = {}".format(precision, error))

Precision = 95.75 % , error = 4925943468.576762


# Calculo de los verdaderos datos a analizar

In [51]:
analizar = pd.read_csv("/home/mati/Desktop/properati_dataset_modificado.csv")

In [52]:
analizar.loc[:,'price_usd'] = analizar.loc[:,columnas].apply(lambda x: bag.predict(x)[0],axis = 1)

In [53]:
resultado = analizar.loc[:,['id','price_usd']]

resultado.to_csv('resultados/Bagging_Regresor_2.csv', index = False)